In [29]:
import json
import pandas as pd
import numpy as np

import re

####  from sqlalchemy import create_engine
####  import psycopg2

# from config import db_password

import time

In [30]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie)                #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles 
    
    return movie

QUESTIONS: #3 are we only suppose to remove TV shows with episodes or also those without Directors and IMDB links?

In [43]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(wiki_data, kaggle_data, rating_data):
    
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv(f'{file_dir}ratings.csv')  

    # Open and read the Wikipedia data JSON file.
    with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)   
    
    # 3. Write a list comprehension to filter out TV shows.
wiki_movies = [movie for movie in wiki_movies_raw
               if 'No. of episodes' not in movie]
                  and 'imdb_link' in movie
                  and ('Director' in movie or 'Directed by' in movie)]    

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
clean_movies = [clean_movie(movie) for movie in wiki_movies]    

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
wiki_movies_df = pd.DataFrame(clean_movies)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        
except: 
        print(f'imdb_id not found.')
        
    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
  

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
box_office = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
form_one = r'\$\d+\.?\d*\s*[mb]illion'
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()
 
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
form_two = r'\$\d{1,3}(?:,\d{3})+'
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()    


    # 12. Add the parse_dollars function.
    
    # This is a function to turn the string of extracted values into a numeric value
    # first extract the data:
    
def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan
    
        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
            
            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            
            # convert to float and multiply by a million
            value = float(s) * 10**6
            
            # return value
            return value
    
        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
            
            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            
            # convert to float and multiply by a billion
            value = float(s) * 10**9
            
            # return value
            return value
    
        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            
            # remove dollar sign and commas
            s = re.sub('\$|,','', s)
            
            # convert to float
            value = float(s)
            
            # return value
            return value
    
        # otherwise, return NaN
        else:
            return np.nan    
    
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    
    # drop all records where the box office amounts are Null
box_office = wiki_movies_df['Box office'].dropna()

    #Create a function to change none string data types to string
def is_not_a_string(x):
    return type(x) != str
box_office[box_office.map(is_not_a_string)]

    # using a lambda expression and join commande to consolidate and convert the box office data
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # create a regular expression string to extractd box office data and count the number of returns with text of m/b illions
form_one = r'\$\d+\.?\d*\s*[mb]illion'
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

    # create a regular expression string to extractd box office data and count the number of returns with no text label
form_two = r'\$\d{1,3}(?:,\d{3})+'
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

    # create a regular expression string to extractd box office that don't match either of the prior two lookups
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

    # this is lambda line to use the two regular expression to get the movies that have either of the box office data
box_office[~matches_form_one & ~matches_form_two]

    # extract from dataframe and apply the parse dollar script above and then drop the Box Office column
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
wiki_movies_df.drop('Box office', axis=1, inplace=True)


    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    
    # drop all records where the budget amounts are Null
budget = wiki_movies_df['Budget'].dropna()

    # using a lambda expression and join commande to consolidate and convert the budget data
budget = budget.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # Remove values between dollar signs an dhyphen for budgets
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

    # create a regular expression string to extractd budget that don't match either of the prior two lookups
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)

    # removing the [] around some numbers in the data
budget = budget.str.replace(r'\[\d+\]\s*', '')

    # extract from dataframe and apply the parse dollar script above and then drop the Budget column
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
wiki_movies_df.drop('Budget', axis=1, inplace=True)



    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    
    # create a variable for release data    
release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

    # parsing out the different date formate to a consistant form
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
date_form_two = r'\d{4}.[01]\d.[0123]\d'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'

    # extract from dataframe and apply the parse dollar script above
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)
 
    
    
    # 16. Clean the running time column in the wiki_movies_df DataFrame.
   
    # create a variable for run time
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

    # determine how many use string boundaries
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False).sum()

    # determine what the other run times look like
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False) != True]

    # determine what the other run times look like
running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False) != True]

    #create the run time extract variable
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

    # Converting the running time to numeric values
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)    #create the run time extract variable

    # Converting hours to minutes so everything will be in minutes for running time to numeric values
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
wiki_movies_df.drop('Running time', axis=1, inplace=True)

    # Return three variables. The first is the wiki_movies_df DataFrame
    
return wiki_movies_df, kaggle_metadata, ratings 

NameError: name 'wiki_movies_raw' is not defined

In [56]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = 'C://Users/13854/Desktop/'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [55]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file, kaggle_file, ratings_file)

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [49]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

TypeError: 'str' object is not callable

In [39]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

AttributeError: 'str' object has no attribute 'columns'